# Modeling Future Sales Data for Shop-Item Pairs

This is a data set from a Kaggle competition representing sales for various physical stores over approximately 2 year period. The items for sale at each store vary by month. 

For this model the following techniques were used:
* encoding total shop sales from previous months as features
* encoding total item sales from previous months as features
* stacking machine learning algorithms 

This code currently gets approximately MSE 1.01 on the test set. At this point, the best way to reduce MSE would be with more feature selection especially text data about the stores and items.

Acknowledgement - some of the data preparation code is from the Coursera-Kaggle machine learning competition course

In [9]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['test.csv.gz', 'sample_submission.csv.gz', 'items.csv', 'item_categories.csv', 'shops.csv', 'sales_train.csv.gz']


In [10]:
from xgboost.sklearn import XGBModel
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import ElasticNet

from sklearn.metrics import mean_squared_error

import itertools
from tqdm import tqdm_notebook

In [11]:
# save memory by downcasting
def downcast_dtypes(df):
    '''
        Changes column types in the dataframe: 
                
                `float64` type to `float32`
                `int64`   type to `int32`
    '''
    
    # Select columns to downcast
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols =   [c for c in df if df[c].dtype == "int64"]
    
    # Downcast
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols]   = df[int_cols].astype(np.int32)
    
    return df

In [12]:
# load data
sales = pd.read_csv('../input/sales_train.csv.gz')
test = pd.read_csv('../input/test.csv.gz')
items = pd.read_csv('../input/items.csv')

In [ ]:
# get all monthly shop/item pairs
index_cols = ['shop_id', 'item_id', 'date_block_num']

grid = []

for block in sales.date_block_num.unique():
    cur_stores = sales.loc[sales.date_block_num == block, 'shop_id'].unique()
    cur_items = sales.loc[sales.date_block_num == block, 'item_id'].unique()
    grid.append(np.array(list(itertools.product(*[cur_stores, cur_items, [block]])), dtype='int32'))
    
grid = pd.DataFrame(np.vstack(grid), columns = index_cols, dtype=np.int32)

In [ ]:
# Groupby data to get shop-item-month aggregates
gb = sales.groupby(index_cols,as_index=False).agg({'item_cnt_day':{'target':'sum'}})
# Fix column names
gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values] 
# Join it to the grid
all_data = pd.merge(grid, gb, how='left', on=index_cols).fillna(0)

# Same as above but with shop-month aggregates
gb = sales.groupby(['shop_id', 'date_block_num'],as_index=False).agg({'item_cnt_day':{'target_shop':'sum'}})
gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values]
all_data = pd.merge(all_data, gb, how='left', on=['shop_id', 'date_block_num']).fillna(0)

# Same as above but with item-month aggregates
gb = sales.groupby(['item_id', 'date_block_num'],as_index=False).agg({'item_cnt_day':{'target_item':'sum'}})
gb.columns = [col[0] if col[-1] == '' else col[-1] for col in gb.columns.values]
all_data = pd.merge(all_data, gb, how='left', on=['item_id', 'date_block_num']).fillna(0)

# Average price for the month for that store, item combination
gb = sales.groupby(['shop_id', 'item_id', 'date_block_num'], as_index=False).agg({'item_price':{'avg_price':'mean'}})
gb.columns = [col[0] if col[-1] == '' else col[-1] for col in gb.columns.values]
all_data = pd.merge(all_data, gb, how='left', on=index_cols).fillna(0)

# Downcast dtypes from 64 to 32 bit to save memory
all_data = downcast_dtypes(all_data)

del grid, gb 

In [ ]:
# calculate lag data

# List of columns that we will use to create lags
cols_to_rename = list(all_data.columns.difference(index_cols)) 

shift_range = [1, 2, 3, 12, 24]

test['date_block_num'] = 34

for month_shift in tqdm_notebook(shift_range):
    train_shift = all_data[index_cols + cols_to_rename].copy()
    
    train_shift['date_block_num'] = train_shift['date_block_num'] + month_shift
    
    foo = lambda x: '{}_lag_{}'.format(x, month_shift) if x in cols_to_rename else x
    train_shift = train_shift.rename(columns=foo)

    # get test data ready
    test_result = pd.merge(test, train_shift, on=index_cols, how='left').fillna(0)
    all_result = pd.merge(all_data, train_shift, on=index_cols, how='left').fillna(0)
    
    all_result = downcast_dtypes(all_result)
    test_result = downcast_dtypes(test_result)
    
    all_result.to_csv('all_' + str(month_shift) + '.csv', index=False)
    test_result.to_csv('test_' + str(month_shift) + '.csv', index=False)
    
    del train_shift, all_result, test_result

In [ ]:
lr1 = LinearRegression()
lr2 = Ridge(normalize=True)
lr3 = ElasticNet(normalize=True)
lr4 = BayesianRidge(normalize=True)
lr5 = LinearRegression()

models = [lr1, lr2, lr3, lr4, lr5]

X_train_level2 = np.array([])
X_test_level2 = np.array([])

for idx, month_shift in tqdm_notebook(enumerate(shift_range)):
    training = pd.read_csv('all_' + str(month_shift) + '.csv')
    testing = pd.read_csv('test_' + str(month_shift) + '.csv')
    model = models[idx]
    
    training = training[training['date_block_num'] >= 12]
    
    # List of all lagged features
    if month_shift == 12:
        fit_cols = [col for col in training.columns if col[-2:] in [str(month_shift)]]
    else:
        fit_cols = [col for col in training.columns if col[-1:] in [str(month_shift)]] 

    # We will drop these at fitting stage
    to_drop_cols = list(set(list(training.columns)) - (set(fit_cols)|set(index_cols))) + ['date_block_num'] 
    
    # Category for each item
    item_category_mapping = items[['item_id','item_category_id']].drop_duplicates()

    training = pd.merge(training, item_category_mapping, how='left', on='item_id')
    training = downcast_dtypes(training)

    testing = pd.merge(testing, item_category_mapping, how='left', on='item_id')
    testing = downcast_dtypes(testing)
    
    # Save `date_block_num`, as we can't use them as features, but will need them to split the dataset into parts 
    dates = training['date_block_num']
    last_block = 34

    dates_train = dates[dates <  last_block]

    X_train = training.drop(to_drop_cols, axis=1)
    X_test =  testing.drop(['date_block_num', 'ID'], axis=1)

    y_train = training['target'].values
    y_test =  None
    
    model.fit(X_train.values, y_train)
    
    pred = model.predict(X_test.values)
    
    if X_test_level2.size == 0:
        X_test_level2 = np.zeros([testing.shape[0], len(shift_range)])
    X_test_level2[:,idx] = pred
        
    print('Train rmse for model is %f' % np.sqrt(mean_squared_error(y_train, model.predict(X_train))))

    # training data for final stacked model
    dates_train_level2 = dates_train[dates_train.isin([27, 28, 29, 30, 31, 32, 33])]

    # That is how we get target for the 2nd level dataset
    y_train_level2 = y_train[dates_train.isin([27, 28, 29, 30, 31, 32, 33])]

    # And here we create 2nd level feature matrix, init it with zeros first
    if X_train_level2.size == 0:
        X_train_level2 = np.zeros([y_train_level2.shape[0], len(shift_range)])

    # Now fill X_train_level2`with metafeatures
    current_row = 0

    for cur_block_num in [24, 25, 26, 27, 28, 29, 30, 31, 32, 33]:

        print(cur_block_num)

        '''
            1. Split `X_train` into parts
               Remember, that corresponding dates are stored in `dates_train` 
            2. Fit linear regression 
            3. Fit LightGBM and put predictions          
            4. Store predictions from 2. and 3. in the right place of `X_train_level2`. 
               You can use `dates_train_level2` for it
               Make sure the order of the meta-features is the same as in `X_test_level2`
        '''      

        #  YOUR CODE GOES HERE
        X_train_part = X_train[dates_train < cur_block_num]
        y_train_part = y_train[dates_train < cur_block_num]

        X_val_part = X_train[dates_train == cur_block_num]

        model.fit(X_train_part, y_train_part)
        pred_train = model.predict(X_val_part)
        
        X_train_level2[current_row:current_row + X_val_part.shape[0], idx] = pred_train

        current_row += X_val_part.shape[0]    
    
    del training, testing, model, X_train, X_test, y_train, y_test

In [ ]:
# fit the level 2 model

level2model = XGBModel(learning_rate=0.6)
level2model.fit(X_train_level2, y_train_level2)

train_preds = level2model.predict(X_train_level2)
rmse_train_stacking = np.sqrt(mean_squared_error(y_train_level2, level2model.predict(X_train_level2)))
print(rmse_train_stacking)
test_preds = level2model.predict(X_test_level2) 

In [ ]:
# output the results to disk

np.savetxt('X_train_level2.csv', X_train_level2, delimiter=',')
np.savetxt('y_train_level2.csv', y_train_level2, delimiter=',')
np.savetxt('X_test_level2.csv', X_test_level2, delimiter=',')
pd.DataFrame({'ID': test.ID, 'item_cnt_month':np.clip(test_preds, 0, 20)}).to_csv('results.csv', index=False)